### Animal Reporter

In [ ]:
class AnimalReporter:
    def __init__(self, animal: str):
        self.animal = animal
    
    def _get_request(self) -> Union[bytes, str]:
        ROOT = 'https://en.wikipedia.org/wiki/'
        r = requests.get(ROOT + self.animal)
        return r.text

    def _get_soup(self, html: Union[bytes, str]) -> BeautifulSoup:
        soup = BeautifulSoup(html, 'html.parser')
        return soup
        
    def get_class(self) -> str:
        html = self._get_request()
        soup = self._get_soup(html)
        class_row = soup.find('td', text = re.compile('Class:'))
        animal_class = class_row.find_next_sibling().text.strip()
        return animal_class
    
    def get_taxonomy(self) -> List:
        html = self._get_request()
        soup = self._get_soup(html)
        syn_text = soup.find('a', text = re.compile('Synonyms'))
        if syn_text:
            syn_header = syn_text.find_parent('tr')
            syn_table = syn_header.find_next_sibling()
            contents = syn_table.find_all('i')
            if contents:
                contents = [x.text for x in contents]
                return contents
        else:
            return []

ar = AnimalReporter('koala')
ar.get_class()

## AsDictMixin

In [1]:
class AsDictionaryMixin:
    def to_dict(self):
        return {
            prop: self._represent(value)
            for prop, value in self.__dict__.items()
            if not self._is_internal(prop)
        }

    def _represent(self, value):
        if isinstance(value, object):
            if hasattr(value, 'to_dict'):
                return value.to_dict()
            else:
                return str(value)
        else:
            return value

    def _is_internal(self, prop):
        return prop.startswith('_')

In [4]:
class Person(AsDictionaryMixin):
    def __init__(self, name, address, salary):
        self.name = name
        self.address = address
        self._salary = salary

ivan = Person('Ivan', 'London', '100000000')
string = "Blair"

In [5]:
ivan.to_dict()

{'name': 'Ivan', 'address': 'London'}